In [2]:
import time
import pandas as pd
import re
import threading
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager

def scrape_products():
    # Configura el ChromeDriver utilizando ChromeDriverManager
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # Lista para almacenar los datos de todos los productos
    all_products_data = []

    # Fecha actual en formato DD-MM-YYYY
    today_date = datetime.now().strftime('%d-%m-%Y')

    # Itera a través de las páginas
    for page in range(1, 20):
        driver.get(f"https://limonada.cl/collections/recien-nacido-4-kids?page={page}")

        # Espera a que carguen los elementos
        WebDriverWait(driver, 1).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[id^="CardLink-"]')))

        # Extrae todos los enlaces de productos en la página actual
        product_links = driver.find_elements(By.CSS_SELECTOR, 'a[id^="CardLink-"]')

        # Itera a través de cada producto en la página
        for index in range(len(product_links)):
            try:
                product_links = driver.find_elements(By.CSS_SELECTOR, 'a[id^="CardLink-"]')
                driver.execute_script("arguments[0].scrollIntoView(true);", product_links[index])
                time.sleep(1)
                driver.execute_script("arguments[0].click();", product_links[index])

                # Espera a que cargue la página del producto
                WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))

                # Extrae la información del producto
                product_name = driver.find_element(By.TAG_NAME, 'h1').text

                # Extrae el nombre de la marca
                try:
                    brand_name = driver.find_element(By.CLASS_NAME, 'product-vendor').text
                except NoSuchElementException:
                    brand_name = ''

                sku_element = driver.find_element(By.XPATH, "//span[contains(text(), 'SKU:')]")
                sku_1 = sku_element.text.split(': ')[1].strip()

                # Extrae precios
                try:
                    original_price = driver.find_element(By.CSS_SELECTOR, 'del.old').text
                except Exception:
                    original_price = driver.find_element(By.CSS_SELECTOR, 'span.regular').text

                try:
                    final_price = driver.find_element(By.CSS_SELECTOR, 'span.new').text
                except Exception:
                    final_price = driver.find_element(By.CSS_SELECTOR, 'span.regular').text

                # Extrae la URL de la imagen
                image_element = driver.find_element(By.XPATH, "//img[@class='img-fluid']")
                image_url = image_element.get_attribute('src')
                if image_url.startswith('http'):
                    image_url = image_url.replace('https:', '//').replace('http:', '//')
                image_url_cleaned = image_url.replace('//limonada', 'limonada', 1)
                image_url_final = re.sub(r'_(.*?)\.jpg', '_1.jpg', image_url_cleaned)

                # Extrae la URL de la página del producto
                product_url = driver.current_url

                # Agrega los datos del producto a la lista
                product_data = {
                    'Nombre': product_name,
                    'Marca': brand_name,
                    'SKU 1': sku_1,
                    'Precio Original': original_price,
                    'Precio Final': final_price,
                    'URL de la Imagen': image_url_final,
                    'URL Página': product_url,
                    'Fecha': today_date
                }

                # Agrega el producto al DataFrame
                all_products_data.append(product_data)

                # Guarda los datos en Excel cada vez que se añade un nuevo producto
                df = pd.DataFrame(all_products_data)
                df.to_excel(r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\CAFE6.xlsx', index=False)

            except ElementClickInterceptedException:
                print(f"Error: Elemento bloqueado para hacer clic en el producto {product_links[index].get_attribute('href')}")
            except NoSuchElementException:
                print(f"Error al extraer datos del producto: {product_links[index].get_attribute('href')}")

            # Regresa a la página anterior
            driver.back()

            # Espera a que cargue la página de productos nuevamente
            WebDriverWait(driver, 1).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[id^="CardLink-"]')))

    # Cierra el navegador
    driver.quit()

# Iniciar el hilo para ejecutar el scraping en segundo plano
scraping_thread = threading.Thread(target=scrape_products)
scraping_thread.start()

# Puedes continuar con otras tareas aquí...
print("El scraping se está ejecutando en segundo plano. Puedes hacer otras tareas.")


El scraping se está ejecutando en segundo plano. Puedes hacer otras tareas.


Exception in thread Thread-6 (scrape_products):
Traceback (most recent call last):
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\constanza.perez\AppData\Local\Temp\ipykernel_2568\83120935.py", line 44, in scrape_products
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\selenium\webdriver\support\wait.py", line 96, in until
    value = method(self._driver)
            ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\selenium\webdriver\support\expected_conditions.py", line 84, in _predicate
    return driver.find_element(*locator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\constanza.perez\AppData\Local\anaconda3\Lib\site-packages\selenium\webdriver\remote\webdriver.py", li